This file allows for testing the GraphWorld setup with GNN implementations.
It is currently set up to test the SSL methods for the JL benchmarker.

Through this notebook you can attach a debugger.
Note that graph_tool does not work on windows, so we cannot use the graph generators.
Instead, we use the standard datasets from PyG.

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from graph_world.self_supervised_learning.benchmarker_jl import NNNodeBenchmarkerJL
from graph_world.models.basic_gnn import GCN
from torch_geometric.datasets import Planetoid

from graph_world.self_supervised_learning.pretext_tasks.auxiliary_property_based import *
from graph_world.self_supervised_learning.pretext_tasks.contrastive_based_different_scale import *

ModuleNotFoundError: No module named 'graphworld'

In [ ]:
# Parameter setup (for cora)
benchmark_params = {
    'epochs' : 50,
    'lr' : 0.01,
    'lambda' : 1
}

h_params = {
    'in_channels' : 1433,
    'hidden_channels' : 16,
    'num_layers' : 2,
    'dropout' : 0.5,
    "embedding_corruption_ratio" : 0.1, 
    "partial_embedding_reconstruction" : True,
    'n_parts': 10,
    'shortest_path_cutoff': 6,
    'N_classes': 4,
    'k_largest': 20
}

generator_config = {
    'num_clusters' : 7,
}

pretext_tasks = [DeepGraphInfomax]

In [ ]:
# Get dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')[0]
# dataset = KarateClub()[0]


In [ ]:
# Training. You can attach a debugger to w/e is needed inside train
benchmarker = NNNodeBenchmarkerJL(generator_config=generator_config, model_class=GCN, 
                benchmark_params=benchmark_params, h_params=h_params, pretext_tasks=pretext_tasks)
# benchmarker.SetMasks(train_mask=dataset.train_mask, val_mask=~dataset.train_mask, test_mask=~dataset.train_mask)
benchmarker.SetMasks(train_mask=dataset.train_mask, val_mask=dataset.val_mask, test_mask=dataset.test_mask)
benchmarker.train(data=dataset, tuning_metric="rocauc_ovr", tuning_metric_is_loss=False)

GCN(1433, 16, num_layers=2)


([2.573589324951172,
  2.4835891723632812,
  2.3994874954223633,
  2.282182216644287,
  2.1765356063842773,
  2.2234225273132324,
  2.1786258220672607,
  2.261712074279785,
  2.0909698009490967,
  2.0249524116516113,
  2.070204734802246,
  1.9639207124710083,
  1.9363347291946411,
  1.922866940498352,
  1.8904564380645752,
  1.7580641508102417,
  1.793889045715332,
  1.6815543174743652,
  1.648980736732483,
  1.6251152753829956,
  1.6130006313323975,
  1.5304874181747437,
  1.4397372007369995,
  1.394917607307434,
  1.2577646970748901,
  1.2471213340759277,
  1.3332464694976807,
  1.1674727201461792,
  1.138140082359314,
  1.2126522064208984,
  1.106448769569397,
  0.9683606028556824,
  1.0529459714889526,
  1.0330358743667603,
  1.0828191041946411,
  1.0333178043365479,
  0.8977891802787781,
  0.8997482061386108,
  0.8376862406730652,
  0.9061324596405029,
  0.759553074836731,
  0.8016982078552246,
  0.762355625629425,
  0.7039662599563599,
  0.7855474948883057,
  0.661488950252533,
 

In [36]:
x = torch.tensor([
    [-1., -2., -3.],
    [1., 2., 3.],
    [0., 0., 0.]
])
x[torch.randperm(x.size()[0]),:]

tensor([[ 1.,  2.,  3.],
        [ 0.,  0.,  0.],
        [-1., -2., -3.]])